In [2]:
import pandas as pd

In [12]:
test = pd.read_csv('artifacts/cbs_model_output.csv')

In [13]:
test.tail()

,recom_date,dataframe,user_film,film_index,recommended_films,recommended_ratings,satisfaction,similarity
4,202305292308,processed_films_202305202210.csv,Avatar,139,"[""Star Wars : Episode V - L'Empire contre-atta...","[5.0, 4.8, 4.4, 4.1, 3.8]",[[1. 0.03413944 0.03774257 ... 0.03580...,Y
5,202305292308,processed_films_202305202210.csv,Coco,27,"['Vice Versa', 'La-haut', 'Soul', 'Your Name',...","[4.7, 4.6, 4.3, 4.0, 3.9]",[[1. 0.03413944 0.03774257 ... 0.03580...,N
6,202305292312,processed_films_202305202210.csv,Forrest Gump,0,"['Barry Lyndon', 'Mirage de la vie', 'Je veux ...","[5.0, 4.3, 4.3, 4.3, 4.2]",[[1. 0.03413944 0.03774257 ... 0.03580...,n
7,202305292319,processed_films_202305202210.csv,Avatar,139,"[""Star Wars : Episode V - L'Empire contre-atta...","[5.0, 4.8, 4.4, 4.1, 3.8]",[[1. 0.03413944 0.03774257 ... 0.03580...,Y
8,202305292321,processed_films_202305202210.csv,Coco,27,"['Vice Versa', 'La-haut', 'Soul', 'Your Name',...","[4.7, 4.6, 4.3, 4.0, 3.9]",[[1. 0.03413944 0.03774257 ... 0.03580...,N
